In [1]:
from sympy.physics.quantum import Commutator, Dagger, Operator
import sympy
from sympy import *
from IPython.display import display
import numpy as np

# Hamiltonian

Define operators and constants

In [2]:
U, F, delta_a, delta_b, J, kappa_a = symbols(r'U F \Delta_a \Delta_b J \kappa_a')

a = Symbol(r'a', commutative=False)
ad = Symbol(r'a^\dagger', commutative=False)
b = Symbol(r'b', commutative=False)
bd = Symbol(r'b^\dagger', commutative=False)

rho = Symbol(r'\rho', commutative=False)

Construct Hamiltonian

In [3]:
Ham1 = J*(ad*b+a*bd) + F*(ad+a) -delta_a*ad*a -delta_b*bd*b #Rational(1, 2)*U*ad*ad*a*a
Ham2 = -delta_a*ad*a -delta_b*bd*b
Ham3 = Mul(U,ad,ad,a,a,evaluate=False)
Ham4 = UnevaluatedExpr(Mul(Rational(1, 2)*U,ad,ad,a,a,evaluate=False))


MyHam = Ham2
display(MyHam)

-\Delta_a*a^\dagger*a - \Delta_b*b^\dagger*b

Jump operators

In [4]:

def getJump(constant,c,rho):
    cd = Dagger(c)
    return UnevaluatedExpr(constant*(cd*rho*c - Rational(1, 2)*(c*cd*rho + rho*c*cd)))

D1 = getJump(kappa_a,a,rho)
display(D1)

    

\kappa_a*(-(\rho*a*Dagger(a) + a*Dagger(a)*\rho)/2 + Dagger(a)*\rho*a)

# Lindblad master equation

In [5]:
ME_comm = -sympy.I* (Commutator(MyHam, rho))+D1

display(ME_comm)
ME= ME_comm.doit()
#display(ME)
ME = expand(ME)
display(ME)

I*[\rho,-\Delta_a*a^\dagger*a - \Delta_b*b^\dagger*b] + \kappa_a*(-(\rho*a*Dagger(a) + a*Dagger(a)*\rho)/2 + Dagger(a)*\rho*a)

-I*\Delta_a*\rho*a^\dagger*a + I*\Delta_a*a^\dagger*a*\rho - I*\Delta_b*\rho*b^\dagger*b + I*\Delta_b*b^\dagger*b*\rho - \kappa_a*\rho*a*Dagger(a)/2 - \kappa_a*a*Dagger(a)*\rho/2 + \kappa_a*Dagger(a)*\rho*a

In [6]:
#ME = ad*a*rho+rho*ad*a

# Convert to desired phase space function

In [7]:
from phasespaceconversion import pow_to_mul_sep2,PhaseSpaceFunction

In [8]:
clean_term = a*ad*ad*rho*ad
display(clean_term)
clean_termNoRho =  clean_term.subs(rho,1 )
print(clean_term.args)

def getPosRho(clean_term,rho):
    counter=0
    items= clean_term.args
    n_items = len(items)
    print(items)
    for k in range(n_items):
        item = items[k]
        if item==rho:
           break    # break here
        else:
            [op, n] = item.as_base_exp()
            counter = counter+n
            print(counter)

    print('rho occurs on position  {}'.format(counter))
    return counter

def labelOperators(ordered_operators, pos_rho):
    n_operators = len(ordered_operators)
    operator_labels = ['RHS']*n_operators
    operator_labels[:pos_rho]=['LHS']*pos_rho
    return operator_labels

pos_rho=getPosRho(clean_term,rho)
ordered_operators = pow_to_mul_sep2(clean_termNoRho)

operator_labels=labelOperators(ordered_operators, pos_rho)

print(ordered_operators)
print(operator_labels)
operator_labels =[] # label each individual operator in the list as RHS or LHS of Rho
print(clean_termNoRho.args)

a*a^\dagger**2*\rho*a^\dagger

(a, a^\dagger**2, \rho, a^\dagger)
(a, a^\dagger**2, \rho, a^\dagger)
1
3
rho occurs on position  3
[a, a^\dagger, a^\dagger, a^\dagger]
['LHS', 'LHS', 'LHS', 'RHS']
(a, a^\dagger**3)


In [9]:
ordered_operators = pow_to_mul_sep2(clean_termNoRho)
display(ordered_operators)



[a, a^\dagger, a^\dagger, a^\dagger]

Wigner

In [10]:
myWigner = PhaseSpaceFunction(ME,'W')
W_eqn = myWigner.getFPfromME()
print('Result')
display(expand(W_eqn))

(\kappa_a*Dagger(a)*\rho*a,
 -\kappa_a*\rho*a*Dagger(a)/2,
 -\kappa_a*a*Dagger(a)*\rho/2,
 I*\Delta_a*a^\dagger*a*\rho,
 I*\Delta_b*b^\dagger*b*\rho,
 -I*\Delta_a*\rho*a^\dagger*a,
 -I*\Delta_b*\rho*b^\dagger*b)

Term 0 :


\kappa_a*Dagger(a)*\rho*a

Dagger(a)*\rho*a

Dagger(a)*a

...

b

RHS term, order operators applied:
[b]


-I*\Delta_b*(\beta*W(\alpha, \alpha^*, \beta, \beta^*) - Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \beta^*)/2)*b^\dagger

Result


I*\Delta_a*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)*a^\dagger + I*\Delta_b*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \beta^*)*b^\dagger - \kappa_a*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)/2

HusimiQ

In [11]:
myQ= PhaseSpaceFunction(ME,'Q')
Q_eqn = myQ.getFPfromME()
print('Result')
display(expand(Q_eqn))

(\kappa_a*Dagger(a)*\rho*a,
 -\kappa_a*\rho*a*Dagger(a)/2,
 -\kappa_a*a*Dagger(a)*\rho/2,
 I*\Delta_a*a^\dagger*a*\rho,
 I*\Delta_b*b^\dagger*b*\rho,
 -I*\Delta_a*\rho*a^\dagger*a,
 -I*\Delta_b*\rho*b^\dagger*b)

Term 0 :


\kappa_a*Dagger(a)*\rho*a

Dagger(a)*\rho*a

Dagger(a)*a

...

b

RHS term, order operators applied:
[b]


-I*\Delta_b*\beta*Q(\alpha, \alpha^*, \beta, \beta^*)*b^\dagger

Result


I*\Delta_a*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)*a^\dagger + I*\Delta_b*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \beta^*)*b^\dagger - \kappa_a*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)/2

P representation

In [12]:
myP= PhaseSpaceFunction(ME,'P')
P_eqn = myP.getFPfromME()
print('Result')
display(expand(P_eqn))

(\kappa_a*Dagger(a)*\rho*a,
 -\kappa_a*\rho*a*Dagger(a)/2,
 -\kappa_a*a*Dagger(a)*\rho/2,
 I*\Delta_a*a^\dagger*a*\rho,
 I*\Delta_b*b^\dagger*b*\rho,
 -I*\Delta_a*\rho*a^\dagger*a,
 -I*\Delta_b*\rho*b^\dagger*b)

Term 0 :


\kappa_a*Dagger(a)*\rho*a

Dagger(a)*\rho*a

Dagger(a)*a

...

b

RHS term, order operators applied:
[b]


-I*\Delta_b*(\beta*P(\alpha, \alpha^*, \beta, \beta^*) - Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \beta^*))*b^\dagger

Result


I*\Delta_a*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)*a^\dagger + I*\Delta_b*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \beta^*)*b^\dagger - \kappa_a*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)/2

# Separate Real and imaginary Parts Wigner

## Not working for now, use mathematica file instead

In [13]:
W_exp_comp = expand(W_eqn)
display(W_exp_comp)

# redefine symbols in original equation (not so nice that this has to be repeated but okay)

#field = Function('W')(alpha,self.alphas,self.beta,self.betas) # can be W, Q or P
alpha = Symbol(r'\alpha', commutative=True)
alphas = Symbol(r'\alpha^*', commutative=True)
beta = Symbol(r'\beta', commutative=True)
betas = Symbol(r'\beta^*', commutative=True)

# define real and imag parts:
alpha_r = Symbol(r'a_R', commutative=True)
alpha_i = Symbol(r'a_I', commutative=True)
beta_r = Symbol(r'b_R', commutative=True)
beta_i = Symbol(r'b_I', commutative=True)

# substitute in expression
W_exp_comp_sep =W_exp_comp.subs(alpha, alpha_r+sympy.I*alpha_i )
W_exp_comp_sep =W_exp_comp.subs(alphas, alpha_r-sympy.I*alpha_i )
W_exp_comp_sep = W_exp_comp.subs(beta, beta_r+sympy.I*beta_i )
W_exp_comp_sep = W_exp_comp.subs(betas, beta_r-sympy.I*beta_i )

display(W_exp_comp_sep)
W_exp_Re = re(W_exp_comp)
display(W_exp_Re)

I*\Delta_a*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)*a^\dagger + I*\Delta_b*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \beta^*)*b^\dagger - \kappa_a*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)/2

I*\Delta_a*Derivative(W(\alpha, \alpha^*, \beta, -I*b_I + b_R), \alpha^*)*a^\dagger + I*\Delta_b*Subs(Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \beta^*), \beta^*, -I*b_I + b_R)*b^\dagger - \kappa_a*Derivative(W(\alpha, \alpha^*, \beta, -I*b_I + b_R), \alpha^*)/2

-re(\kappa_a*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*))/2 - im(\Delta_a*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)*a^\dagger) - im(\Delta_b*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \beta^*)*b^\dagger)

In [14]:
W_exp_comp = expand(W_eqn)
W_exp_comp.atoms()
W_exp_comp.atoms(Function)

{W(\alpha, \alpha^*, \beta, \beta^*)}

In [15]:
latex(W_exp_comp)

'i \\Delta_{a} \\frac{\\partial}{\\partial \\alpha^{*}} W{\\left(\\alpha,\\alpha^{*},\\beta,\\beta^{*} \\right)} a^{\\dagger} + i \\Delta_{b} \\frac{\\partial}{\\partial \\beta^{*}} W{\\left(\\alpha,\\alpha^{*},\\beta,\\beta^{*} \\right)} b^{\\dagger} - \\frac{\\kappa_{a} \\frac{\\partial}{\\partial \\alpha^{*}} W{\\left(\\alpha,\\alpha^{*},\\beta,\\beta^{*} \\right)}}{2}'